In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

In [3]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
import multiprocessing

In [4]:
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.ensemble import BaggingClassifier

In [5]:
from xgboost import XGBClassifier

In [6]:
labels = pd.read_csv('../../csv/train_labels.csv')
labels.head()

,building_id,damage_grade
0,802906,3
1,28830,2
2,94947,3
3,590882,2
4,201944,3


In [7]:
values = pd.read_csv('../../csv/train_values.csv')
values.T

,0,1,2,3,4,5,6,7,8,9,...,260591,260592,260593,260594,260595,260596,260597,260598,260599,260600
building_id,802906,28830,94947,590882,201944,333020,728451,475515,441126,989500,...,560805,207683,226421,159555,827012,688636,669485,602512,151409,747594
geo_level_1_id,6,8,21,22,11,8,9,20,0,26,...,20,10,8,27,8,25,17,17,26,21
geo_level_2_id,487,900,363,418,131,558,475,323,757,886,...,368,1382,767,181,268,1335,715,51,39,9
geo_level_3_id,12198,2812,8973,10694,1488,6089,12066,12236,7219,994,...,5980,1903,8613,1537,4718,1621,2060,8163,1851,9101
count_floors_pre_eq,2,2,2,2,3,2,2,2,2,1,...,1,2,2,6,2,1,2,3,2,3
age,30,10,10,10,30,10,25,0,15,0,...,25,25,5,0,20,55,0,55,10,10
area_percentage,6,8,5,6,8,9,3,8,8,13,...,5,5,13,13,8,6,6,6,14,7
height_percentage,5,7,5,5,9,5,4,6,6,4,...,3,5,5,12,5,3,5,7,6,6
land_surface_condition,t,o,t,t,t,t,n,t,t,t,...,n,t,t,t,t,n,t,t,t,n
foundation_type,r,r,r,r,r,r,r,w,r,i,...,r,r,r,r,r,r,r,r,r,r


In [8]:
to_be_categorized = ["land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for row in to_be_categorized:
    values[row] = values[row].astype("category")
values.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   building_id                             260601 non-null  int64   
 1   geo_level_1_id                          260601 non-null  int64   
 2   geo_level_2_id                          260601 non-null  int64   
 3   geo_level_3_id                          260601 non-null  int64   
 4   count_floors_pre_eq                     260601 non-null  int64   
 5   age                                     260601 non-null  int64   
 6   area_percentage                         260601 non-null  int64   
 7   height_percentage                       260601 non-null  int64   
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                       

In [9]:
datatypes = dict(values.dtypes)
for row in values.columns:
    if datatypes[row] != "int64" and datatypes[row] != "int32" and \
       datatypes[row] != "int16" and datatypes[row] != "int8":
        continue
    if values[row].nlargest(1).item() > 32767 and values[row].nlargest(1).item() < 2**31:
        values[row] = values[row].astype(np.int32)
    elif values[row].nlargest(1).item() > 127:
        values[row] = values[row].astype(np.int16)
    else:
        values[row] = values[row].astype(np.int8)

In [10]:
labels["building_id"] = labels["building_id"].astype(np.int32)
labels["damage_grade"] = labels["damage_grade"].astype(np.int8)
labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260601 entries, 0 to 260600
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   building_id   260601 non-null  int32
 1   damage_grade  260601 non-null  int8 
dtypes: int32(1), int8(1)
memory usage: 1.2 MB


# Feature Engineering para XGBoost

In [11]:
important_values = values\
                .merge(labels, on="building_id")
important_values.drop(columns=["building_id"], inplace = True)
important_values["geo_level_1_id"] = important_values["geo_level_1_id"].astype("category")
important_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
1,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
2,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
3,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
4,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,25,1335,1621,1,55,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,2
260597,17,715,2060,2,0,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
260598,17,51,8163,3,55,6,7,t,r,q,...,0,0,0,0,0,0,0,0,0,3
260599,26,39,1851,2,10,14,6,t,r,x,...,0,0,0,0,0,0,0,0,0,2


In [12]:

X_train, X_test, y_train, y_test = train_test_split(important_values.drop(columns = 'damage_grade'),
                                                    important_values['damage_grade'], test_size = 0.01, random_state = 123)

In [13]:
#OneHotEncoding
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    X_train = encode_and_bind(X_train, feature)
    X_test = encode_and_bind(X_test, feature)

In [14]:
X_train

,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
236915,1160,3465,1,15,6,4,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
62895,1394,4411,2,30,8,5,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
45664,18,1389,2,5,8,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
122801,407,6400,2,10,9,6,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
75248,613,8037,2,10,13,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192476,217,10644,1,25,4,6,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
17730,600,4813,2,20,13,8,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
28030,463,4692,2,10,9,4,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
15725,600,157,2,50,5,8,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


# Entreno tres de los mejores modelos con Voting.

In [15]:
xgb_model_1 = XGBClassifier(n_estimators = 350,
                          subsample = 0.885,
                          booster = 'gbtree',
                          gamma = 1,
                          learning_rate = 0.45,
                          label_encoder = False,
                          verbosity = 2)

bc_xgb_model_1 = BaggingClassifier(base_estimator = xgb_model_1,
                        n_estimators = 5,
                        oob_score = True,
                        bootstrap_features = True,
                        random_state = 0,
                        verbose = 3)


bc_xgb_model_1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/joaquinfontela/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Building estimator 1 of 5 for this parallel run (total 5)...
[13:23:25] WARNING: ../src/learner.cc:573: 
Parameters: { "label_encoder" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:23:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 122 extra nodes, 2 pruned nodes, max_depth=6
[13:23:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 118 extra nodes, 0 pruned nodes, max_depth=6
[13:23:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 126 extra nodes, 0 pruned nodes, max_depth=6
[13:23:26] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[13:23:26

[13:23:38] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 18 pruned nodes, max_depth=6
[13:23:38] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 100 extra nodes, 6 pruned nodes, max_depth=6
[13:23:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 106 extra nodes, 4 pruned nodes, max_depth=6
[13:23:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 66 extra nodes, 14 pruned nodes, max_depth=6
[13:23:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 102 extra nodes, 10 pruned nodes, max_depth=6
[13:23:39] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 98 extra nodes, 4 pruned nodes, max_depth=6
[13:23:40] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 106 extra nodes, 10 pruned nodes, max_depth=6
[13:23:40] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 102 extra nodes, 10 pruned nodes, max_depth=6
[13:23:40] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 60 extra nodes, 6

[13:24:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 118 extra nodes, 2 pruned nodes, max_depth=6
[13:24:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 108 extra nodes, 10 pruned nodes, max_depth=6
[13:24:01] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 86 extra nodes, 4 pruned nodes, max_depth=6
[13:24:02] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 70 extra nodes, 24 pruned nodes, max_depth=6
[13:24:02] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 94 extra nodes, 6 pruned nodes, max_depth=6
[13:24:02] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 52 extra nodes, 10 pruned nodes, max_depth=6
[13:24:02] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 6 pruned nodes, max_depth=6
[13:24:03] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 66 extra nodes, 6 pruned nodes, max_depth=6
[13:24:03] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 62 extra nodes, 8 prun

[13:24:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 116 extra nodes, 2 pruned nodes, max_depth=6
[13:24:21] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 116 extra nodes, 6 pruned nodes, max_depth=6
[13:24:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 66 extra nodes, 4 pruned nodes, max_depth=6
[13:24:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 104 extra nodes, 14 pruned nodes, max_depth=6
[13:24:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 120 extra nodes, 4 pruned nodes, max_depth=6
[13:24:22] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 78 extra nodes, 18 pruned nodes, max_depth=6
[13:24:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 98 extra nodes, 12 pruned nodes, max_depth=6
[13:24:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 100 extra nodes, 18 pruned nodes, max_depth=6
[13:24:23] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 84 extra nodes, 4 

...

[13:45:49] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 22 pruned nodes, max_depth=6
[13:45:49] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 60 extra nodes, 22 pruned nodes, max_depth=6
[13:45:49] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 36 extra nodes, 18 pruned nodes, max_depth=6
[13:45:50] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 52 extra nodes, 32 pruned nodes, max_depth=6
[13:45:50] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 42 extra nodes, 24 pruned nodes, max_depth=6
[13:45:50] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 46 extra nodes, 24 pruned nodes, max_depth=6
[13:45:50] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 58 extra nodes, 22 pruned nodes, max_depth=6
[13:45:51] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 22 pruned nodes, max_depth=6
[13:45:51] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 70 extra nodes, 22 

[13:46:08] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 64 extra nodes, 18 pruned nodes, max_depth=6
[13:46:08] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 38 pruned nodes, max_depth=6
[13:46:08] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 28 pruned nodes, max_depth=6
[13:46:08] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 60 extra nodes, 22 pruned nodes, max_depth=6
[13:46:09] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 82 extra nodes, 20 pruned nodes, max_depth=6
[13:46:09] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 56 extra nodes, 34 pruned nodes, max_depth=6
[13:46:09] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 48 extra nodes, 28 pruned nodes, max_depth=6
[13:46:09] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 68 extra nodes, 18 pruned nodes, max_depth=6
[13:46:10] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 58 extra nodes, 36 

[13:46:26] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 70 extra nodes, 12 pruned nodes, max_depth=6
[13:46:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 52 pruned nodes, max_depth=6
[13:46:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 60 extra nodes, 30 pruned nodes, max_depth=6
[13:46:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 48 extra nodes, 16 pruned nodes, max_depth=6
[13:46:27] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 72 extra nodes, 18 pruned nodes, max_depth=6
[13:46:28] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 56 extra nodes, 20 pruned nodes, max_depth=6
[13:46:28] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 40 extra nodes, 40 pruned nodes, max_depth=6
[13:46:28] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 70 extra nodes, 16 pruned nodes, max_depth=6
[13:46:28] INFO: ../src/tree/updater_prune.cc:101: tree pruning end, 52 extra nodes, 16 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 23.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 23.1min finished
/home/joaquinfontela/.local/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:640: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/joaquinfontela/.local/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:644: RuntimeWarning: invalid value encountered in true_divide
  oob_decision_function = (predictions /


BaggingClassifier(base_estimator=XGBClassifier(base_score=None,
                                               booster='gbtree',
                                               colsample_bylevel=None,
                                               colsample_bynode=None,
                                               colsample_bytree=None, gamma=1,
                                               gpu_id=None,
                                               importance_type='gain',
                                               interaction_constraints=None,
                                               label_encoder=False,
                                               learning_rate=0.45,
                                               max_delta_step=None,
                                               max_depth=None,
                                               min_child_weight=None,
                                               missing=nan,
                                               

In [16]:
# xgb_model_2 = XGBClassifier(n_estimators = 350,
#                           subsample = 0.950,
#                           booster = 'gbtree',
#                           gamma = 0.5,
#                           learning_rate = 0.45,
#                           label_encoder = False,
#                           verbosity = 2)
# xgb_model_2.fit(X_train, y_train)

In [17]:
# xgb_model_3 = XGBClassifier(n_estimators = 350,
#                           subsample = 0.750,
#                           booster = 'gbtree',
#                           gamma = 1,
#                           learning_rate = 0.45,
#                           label_encoder = False,
#                           verbosity = 2)
# xgb_model_3.fit(X_train, y_train)

In [18]:
# xgb_model_4 = XGBClassifier(n_estimators = 350,
#                           subsample = 0.80,
#                           booster = 'gbtree',
#                           gamma = 1,
#                           learning_rate = 0.55,
#                           label_encoder = False,
#                           verbosity = 2)
# xgb_model_4.fit(X_train, y_train)

In [19]:
rf_model_1 = RandomForestClassifier(n_estimators = 150,
                                  max_depth = None,
                                  max_features = 45,
                                  min_samples_split = 15,
                                  min_samples_leaf = 1,
                                  criterion = "gini",
                                  verbose=True)
rf_model_1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:  2.8min finished


RandomForestClassifier(max_features=45, min_samples_split=15, n_estimators=150,
                       verbose=True)

In [20]:
# rf_model_2 = RandomForestClassifier(n_estimators = 250,
#                                   max_depth = None,
#                                   max_features = 45,
#                                   min_samples_split = 15,
#                                   min_samples_leaf = 1,
#                                   criterion = "gini",
#                                   verbose=True,
#                                   n_jobs =-1)
# rf_model_2.fit(X_train, y_train)

In [21]:
import lightgbm as lgb
lgbm_model_1 = lgb.LGBMClassifier(boosting_type='gbdt', 
                         colsample_bytree=1.0,
                         importance_type='split', 
                         learning_rate=0.15,
                         max_depth=None,
                         n_estimators=1600, 
                         n_jobs=-1,
                         objective=None,
                         subsample=1.0, 
                         subsample_for_bin=200000, 
                         subsample_freq=0)

bc_lgbm_model_1 = BaggingClassifier(base_estimator = lgbm_model_1,
                        n_estimators = 20,
                        oob_score = True,
                        bootstrap_features = True,
                        random_state = 0,
                        verbose = 3)


bc_lgbm_model_1.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Building estimator 1 of 20 for this parallel run (total 20)...
Building estimator 2 of 20 for this parallel run (total 20)...
Building estimator 3 of 20 for this parallel run (total 20)...
Building estimator 4 of 20 for this parallel run (total 20)...
Building estimator 5 of 20 for this parallel run (total 20)...
Building estimator 6 of 20 for this parallel run (total 20)...
Building estimator 7 of 20 for this parallel run (total 20)...
Building estimator 8 of 20 for this parallel run (total 20)...
Building estimator 9 of 20 for this parallel run (total 20)...
Building estimator 10 of 20 for this parallel run (total 20)...
Building estimator 11 of 20 for this parallel run (total 20)...
Building estimator 12 of 20 for this parallel run (total 20)...
Building estimator 13 of 20 for this parallel run (total 20)...
Building estimator 14 of 20 for this parallel run (total 20)...
Building estimator 15 of 20 for this parallel run (total 20)...
Building estimator 16 of 20 for this parallel run

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  9.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  9.5min finished
/home/joaquinfontela/.local/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:640: UserWarning: Some inputs do not have OOB scores. This probably means too few estimators were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/home/joaquinfontela/.local/lib/python3.8/site-packages/sklearn/ensemble/_bagging.py:644: RuntimeWarning: invalid value encountered in true_divide
  oob_decision_function = (predictions /


BaggingClassifier(base_estimator=LGBMClassifier(learning_rate=0.15,
                                                max_depth=None,
                                                n_estimators=1600),
                  bootstrap_features=True, n_estimators=20, oob_score=True,
                  random_state=0, verbose=3)

In [22]:
# lgbm_model_2 = lgb.LGBMClassifier(boosting_type='gbdt', 
#                          colsample_bytree=1.0,
#                          importance_type='split', 
#                          learning_rate=0.15,
#                          max_depth=25,
#                          n_estimators=1750, 
#                          n_jobs=-1,
#                          objective=None,
#                          subsample=0.7, 
#                          subsample_for_bin=240000, 
#                          subsample_freq=0)
# lgbm_model_2.fit(X_train, y_train)

In [23]:
# lgbm_model_3 = lgb.LGBMClassifier(boosting_type='gbdt', 
#                          colsample_bytree=1.0,
#                          importance_type='split', 
#                          learning_rate=0.20,
#                          max_depth=40,
#                          n_estimators=1450, 
#                          n_jobs=-1,
#                          objective=None,
#                          subsample=0.7, 
#                          subsample_for_bin=160000, 
#                          subsample_freq=0)
# lgbm_model_3.fit(X_train, y_train)

In [24]:
from catboost import CatBoostClassifier
cb_model_1 = CatBoostClassifier(iterations = 7500,
                                eval_metric='Accuracy',
                                loss_function='MultiClass',
                                use_best_model=True,
                                random_seed=1,
                                max_depth = None,
                                boost_from_average = False,
                                verbose=True)
cb_model_1.fit(X_train, y_train, 
          cat_features=['geo_level_2_id', 'geo_level_3_id'],
          eval_set=(X_train,y_train))

0:	learn: 0.6928494	test: 0.6987876	best: 0.6987876 (0)	total: 266ms	remaining: 33m 12s
1:	learn: 0.6986558	test: 0.7037916	best: 0.7037916 (1)	total: 403ms	remaining: 25m 10s
2:	learn: 0.6998767	test: 0.7046908	best: 0.7046908 (2)	total: 526ms	remaining: 21m 54s
3:	learn: 0.7022334	test: 0.7070513	best: 0.7070513 (3)	total: 629ms	remaining: 19m 39s
4:	learn: 0.7023535	test: 0.7072606	best: 0.7072606 (4)	total: 798ms	remaining: 19m 55s
5:	learn: 0.7029776	test: 0.7078227	best: 0.7078227 (5)	total: 932ms	remaining: 19m 23s
6:	learn: 0.7027063	test: 0.7076715	best: 0.7078227 (5)	total: 1.08s	remaining: 19m 15s
7:	learn: 0.7028497	test: 0.7078033	best: 0.7078227 (5)	total: 1.24s	remaining: 19m 25s
8:	learn: 0.7034543	test: 0.7083924	best: 0.7083924 (8)	total: 1.43s	remaining: 19m 52s
9:	learn: 0.7032567	test: 0.7083188	best: 0.7083924 (8)	total: 1.58s	remaining: 19m 46s
10:	learn: 0.7035745	test: 0.7086056	best: 0.7086056 (10)	total: 1.74s	remaining: 19m 43s
11:	learn: 0.7037295	test: 0.7

93:	learn: 0.7308426	test: 0.7469050	best: 0.7469050 (93)	total: 15.4s	remaining: 20m 9s
94:	learn: 0.7308620	test: 0.7469321	best: 0.7469321 (94)	total: 15.6s	remaining: 20m 16s
95:	learn: 0.7309782	test: 0.7470561	best: 0.7470561 (95)	total: 15.8s	remaining: 20m 14s
96:	learn: 0.7312031	test: 0.7473856	best: 0.7473856 (96)	total: 15.9s	remaining: 20m 15s
97:	learn: 0.7312496	test: 0.7474476	best: 0.7474476 (97)	total: 16.1s	remaining: 20m 17s
98:	learn: 0.7314085	test: 0.7476530	best: 0.7476530 (98)	total: 16.3s	remaining: 20m 18s
99:	learn: 0.7315441	test: 0.7477344	best: 0.7477344 (99)	total: 16.4s	remaining: 20m 16s
100:	learn: 0.7315209	test: 0.7478120	best: 0.7478120 (100)	total: 16.6s	remaining: 20m 18s
101:	learn: 0.7316100	test: 0.7478391	best: 0.7478391 (101)	total: 16.8s	remaining: 20m 21s
102:	learn: 0.7316953	test: 0.7480445	best: 0.7480445 (102)	total: 17s	remaining: 20m 22s
103:	learn: 0.7317690	test: 0.7481337	best: 0.7481337 (103)	total: 17.2s	remaining: 20m 21s
104:	

184:	learn: 0.7358427	test: 0.7528315	best: 0.7528315 (184)	total: 32.5s	remaining: 21m 25s
185:	learn: 0.7358504	test: 0.7529477	best: 0.7529477 (185)	total: 32.7s	remaining: 21m 26s
186:	learn: 0.7359280	test: 0.7529904	best: 0.7529904 (186)	total: 32.9s	remaining: 21m 27s
187:	learn: 0.7359551	test: 0.7530446	best: 0.7530446 (187)	total: 33.1s	remaining: 21m 27s
188:	learn: 0.7359900	test: 0.7531183	best: 0.7531183 (188)	total: 33.3s	remaining: 21m 27s
189:	learn: 0.7360016	test: 0.7531493	best: 0.7531493 (189)	total: 33.5s	remaining: 21m 29s
190:	learn: 0.7359977	test: 0.7531958	best: 0.7531958 (190)	total: 33.7s	remaining: 21m 28s
191:	learn: 0.7361412	test: 0.7532307	best: 0.7532307 (191)	total: 33.8s	remaining: 21m 27s
192:	learn: 0.7362264	test: 0.7532501	best: 0.7532501 (192)	total: 34s	remaining: 21m 28s
193:	learn: 0.7362264	test: 0.7532811	best: 0.7532811 (193)	total: 34.2s	remaining: 21m 28s
194:	learn: 0.7363350	test: 0.7533896	best: 0.7533896 (194)	total: 34.4s	remaining

274:	learn: 0.7385792	test: 0.7559943	best: 0.7559943 (274)	total: 50.2s	remaining: 21m 58s
275:	learn: 0.7386722	test: 0.7560176	best: 0.7560176 (275)	total: 50.3s	remaining: 21m 57s
276:	learn: 0.7386528	test: 0.7560680	best: 0.7560680 (276)	total: 50.5s	remaining: 21m 57s
277:	learn: 0.7386877	test: 0.7560951	best: 0.7560951 (277)	total: 50.7s	remaining: 21m 58s
278:	learn: 0.7386567	test: 0.7560912	best: 0.7560951 (277)	total: 51s	remaining: 21m 58s
279:	learn: 0.7386916	test: 0.7561029	best: 0.7561029 (279)	total: 51.2s	remaining: 21m 59s
280:	learn: 0.7386838	test: 0.7561532	best: 0.7561532 (280)	total: 51.4s	remaining: 22m
281:	learn: 0.7387381	test: 0.7561571	best: 0.7561571 (281)	total: 51.6s	remaining: 22m
282:	learn: 0.7387536	test: 0.7561571	best: 0.7561571 (281)	total: 51.8s	remaining: 22m
283:	learn: 0.7387691	test: 0.7561881	best: 0.7561881 (283)	total: 52s	remaining: 22m 1s
284:	learn: 0.7388428	test: 0.7562114	best: 0.7562114 (284)	total: 52.2s	remaining: 22m 2s
285:	l

365:	learn: 0.7405909	test: 0.7579362	best: 0.7579634 (364)	total: 1m 8s	remaining: 22m 9s
366:	learn: 0.7406102	test: 0.7579634	best: 0.7579634 (364)	total: 1m 8s	remaining: 22m 10s
367:	learn: 0.7406296	test: 0.7580099	best: 0.7580099 (367)	total: 1m 8s	remaining: 22m 11s
368:	learn: 0.7406994	test: 0.7580641	best: 0.7580641 (368)	total: 1m 8s	remaining: 22m 11s
369:	learn: 0.7406219	test: 0.7580913	best: 0.7580913 (369)	total: 1m 9s	remaining: 22m 11s
370:	learn: 0.7406258	test: 0.7580448	best: 0.7580913 (369)	total: 1m 9s	remaining: 22m 12s
371:	learn: 0.7406606	test: 0.7580990	best: 0.7580990 (371)	total: 1m 9s	remaining: 22m 12s
372:	learn: 0.7406723	test: 0.7581378	best: 0.7581378 (372)	total: 1m 9s	remaining: 22m 12s
373:	learn: 0.7406955	test: 0.7581455	best: 0.7581455 (373)	total: 1m 9s	remaining: 22m 12s
374:	learn: 0.7406684	test: 0.7581765	best: 0.7581765 (374)	total: 1m 10s	remaining: 22m 13s
375:	learn: 0.7407575	test: 0.7581572	best: 0.7581765 (374)	total: 1m 10s	remain

...

7101:	learn: 0.7810569	test: 0.7875532	best: 0.7875920 (7096)	total: 23m 41s	remaining: 1m 19s
7102:	learn: 0.7810879	test: 0.7875338	best: 0.7875920 (7096)	total: 23m 41s	remaining: 1m 19s
7103:	learn: 0.7810879	test: 0.7875416	best: 0.7875920 (7096)	total: 23m 42s	remaining: 1m 19s
7104:	learn: 0.7810918	test: 0.7875377	best: 0.7875920 (7096)	total: 23m 42s	remaining: 1m 19s
7105:	learn: 0.7810724	test: 0.7875454	best: 0.7875920 (7096)	total: 23m 42s	remaining: 1m 18s
7106:	learn: 0.7810879	test: 0.7875610	best: 0.7875920 (7096)	total: 23m 42s	remaining: 1m 18s
7107:	learn: 0.7811073	test: 0.7875726	best: 0.7875920 (7096)	total: 23m 42s	remaining: 1m 18s
7108:	learn: 0.7810957	test: 0.7875610	best: 0.7875920 (7096)	total: 23m 43s	remaining: 1m 18s
7109:	learn: 0.7810996	test: 0.7875648	best: 0.7875920 (7096)	total: 23m 43s	remaining: 1m 18s
7110:	learn: 0.7811228	test: 0.7875881	best: 0.7875920 (7096)	total: 23m 43s	remaining: 1m 17s
7111:	learn: 0.7811151	test: 0.7875726	best: 0.787

7189:	learn: 0.7813941	test: 0.7878129	best: 0.7878439 (7186)	total: 23m 59s	remaining: 1m 2s
7190:	learn: 0.7813825	test: 0.7877935	best: 0.7878439 (7186)	total: 23m 59s	remaining: 1m 1s
7191:	learn: 0.7814368	test: 0.7878051	best: 0.7878439 (7186)	total: 23m 59s	remaining: 1m 1s
7192:	learn: 0.7814290	test: 0.7878206	best: 0.7878439 (7186)	total: 23m 59s	remaining: 1m 1s
7193:	learn: 0.7814290	test: 0.7878206	best: 0.7878439 (7186)	total: 24m	remaining: 1m 1s
7194:	learn: 0.7814329	test: 0.7878090	best: 0.7878439 (7186)	total: 24m	remaining: 1m 1s
7195:	learn: 0.7814407	test: 0.7878323	best: 0.7878439 (7186)	total: 24m	remaining: 1m
7196:	learn: 0.7814251	test: 0.7878090	best: 0.7878439 (7186)	total: 24m	remaining: 1m
7197:	learn: 0.7814407	test: 0.7877741	best: 0.7878439 (7186)	total: 24m 1s	remaining: 1m
7198:	learn: 0.7814407	test: 0.7877703	best: 0.7878439 (7186)	total: 24m 1s	remaining: 1m
7199:	learn: 0.7814523	test: 0.7878129	best: 0.7878439 (7186)	total: 24m 1s	remaining: 1m


7278:	learn: 0.7817197	test: 0.7879563	best: 0.7879989 (7262)	total: 24m 17s	remaining: 44.2s
7279:	learn: 0.7817081	test: 0.7879369	best: 0.7879989 (7262)	total: 24m 17s	remaining: 44s
7280:	learn: 0.7816926	test: 0.7879524	best: 0.7879989 (7262)	total: 24m 17s	remaining: 43.8s
7281:	learn: 0.7816732	test: 0.7879524	best: 0.7879989 (7262)	total: 24m 17s	remaining: 43.6s
7282:	learn: 0.7816965	test: 0.7879292	best: 0.7879989 (7262)	total: 24m 17s	remaining: 43.4s
7283:	learn: 0.7817159	test: 0.7879369	best: 0.7879989 (7262)	total: 24m 18s	remaining: 43.2s
7284:	learn: 0.7817197	test: 0.7879369	best: 0.7879989 (7262)	total: 24m 18s	remaining: 43s
7285:	learn: 0.7817275	test: 0.7879408	best: 0.7879989 (7262)	total: 24m 18s	remaining: 42.8s
7286:	learn: 0.7817352	test: 0.7879486	best: 0.7879989 (7262)	total: 24m 18s	remaining: 42.6s
7287:	learn: 0.7817275	test: 0.7879369	best: 0.7879989 (7262)	total: 24m 18s	remaining: 42.4s
7288:	learn: 0.7817469	test: 0.7879524	best: 0.7879989 (7262)	to

7366:	learn: 0.7821228	test: 0.7881230	best: 0.7881811 (7335)	total: 24m 35s	remaining: 26.6s
7367:	learn: 0.7821538	test: 0.7881152	best: 0.7881811 (7335)	total: 24m 35s	remaining: 26.4s
7368:	learn: 0.7821151	test: 0.7881230	best: 0.7881811 (7335)	total: 24m 35s	remaining: 26.2s
7369:	learn: 0.7821500	test: 0.7881462	best: 0.7881811 (7335)	total: 24m 35s	remaining: 26s
7370:	learn: 0.7821732	test: 0.7881462	best: 0.7881811 (7335)	total: 24m 36s	remaining: 25.8s
7371:	learn: 0.7821771	test: 0.7881966	best: 0.7881966 (7371)	total: 24m 36s	remaining: 25.6s
7372:	learn: 0.7821771	test: 0.7881966	best: 0.7881966 (7371)	total: 24m 36s	remaining: 25.4s
7373:	learn: 0.7821694	test: 0.7881734	best: 0.7881966 (7371)	total: 24m 36s	remaining: 25.2s
7374:	learn: 0.7821771	test: 0.7881772	best: 0.7881966 (7371)	total: 24m 36s	remaining: 25s
7375:	learn: 0.7821849	test: 0.7881966	best: 0.7881966 (7371)	total: 24m 36s	remaining: 24.8s
7376:	learn: 0.7821849	test: 0.7882044	best: 0.7882044 (7376)	to

7455:	learn: 0.7826422	test: 0.7884524	best: 0.7884912 (7450)	total: 24m 53s	remaining: 8.81s
7456:	learn: 0.7826577	test: 0.7884602	best: 0.7884912 (7450)	total: 24m 53s	remaining: 8.61s
7457:	learn: 0.7826732	test: 0.7884912	best: 0.7884912 (7450)	total: 24m 53s	remaining: 8.41s
7458:	learn: 0.7826965	test: 0.7884524	best: 0.7884912 (7450)	total: 24m 53s	remaining: 8.21s
7459:	learn: 0.7826422	test: 0.7884369	best: 0.7884912 (7450)	total: 24m 54s	remaining: 8.01s
7460:	learn: 0.7826345	test: 0.7884873	best: 0.7884912 (7450)	total: 24m 54s	remaining: 7.81s
7461:	learn: 0.7826229	test: 0.7884951	best: 0.7884951 (7461)	total: 24m 54s	remaining: 7.61s
7462:	learn: 0.7826112	test: 0.7885067	best: 0.7885067 (7462)	total: 24m 54s	remaining: 7.41s
7463:	learn: 0.7826422	test: 0.7884912	best: 0.7885067 (7462)	total: 24m 54s	remaining: 7.21s
7464:	learn: 0.7826384	test: 0.7884912	best: 0.7885067 (7462)	total: 24m 55s	remaining: 7.01s
7465:	learn: 0.7826384	test: 0.7885300	best: 0.7885300 (7465

In [25]:
# import sklearn as sk
# import sklearn.neural_network
# neuronal_1 = sk.neural_network.MLPClassifier(solver='adam', 
#                                      activation = 'relu',
#                                      learning_rate_init=0.001,
#                                      learning_rate = 'adaptive',
#                                      verbose=True,
#                                      batch_size = 'auto')
# neuronal_1.fit(X_train, y_train)

In [26]:
gb_model_1 = GradientBoostingClassifier(n_estimators = 300,
                                  max_depth = 9,
                                  min_samples_split = 2,
                                  min_samples_leaf = 3,
                                  subsample=0.6,
                                  verbose=True,
                                  learning_rate=0.15)
gb_model_1.fit(X_train, y_train)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.8416           0.0684           15.89m
         2           0.7971           0.0413           15.75m
         3           0.7667           0.0291           15.69m
         4           0.7445           0.0214           15.69m
         5           0.7255           0.0164           15.69m
         6           0.7127           0.0133           15.67m
         7           0.6984           0.0108           15.64m
         8           0.6894           0.0088           15.68m
         9           0.6808           0.0070           15.63m
        10           0.6741           0.0061           15.54m
        20           0.6256           0.0020           15.12m
        30           0.5971           0.0011           14.61m
        40           0.5759           0.0006           14.06m
        50           0.5584           0.0006           13.49m
        60           0.5444           0.0007           12.92m
       

GradientBoostingClassifier(learning_rate=0.15, max_depth=9, min_samples_leaf=3,
                           n_estimators=300, subsample=0.6, verbose=True)

In [27]:
eclf_model = EnsembleVoteClassifier(clfs=[bc_xgb_model_1,
                                          rf_model_1,
                                          bc_lgbm_model_1,
                                          cb_model_1,
                                          gb_model_1],
                                    weights=[3,1,3,6,1.5],
                                    fit_base_estimators=False,
                                    voting='soft')

In [28]:
eclf_model.fit(X_train, y_train)

/home/joaquinfontela/.local/lib/python3.8/site-packages/mlxtend/classifier/ensemble_vote.py:166: UserWarning: fit_base_estimators=False enforces use_clones to be `False`
  warnings.warn("fit_base_estimators=False "


EnsembleVoteClassifier(clfs=[BaggingClassifier(base_estimator=XGBClassifier(base_score=None,
                                                                            booster='gbtree',
                                                                            colsample_bylevel=None,
                                                                            colsample_bynode=None,
                                                                            colsample_bytree=None,
                                                                            gamma=1,
                                                                            gpu_id=None,
                                                                            importance_type='gain',
                                                                            interaction_constraints=None,
                                                                            label_encoder=False,
                                      

In [29]:
y_preds = eclf_model.predict(X_test)
f1_score(y_test, y_preds, average='micro')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.0s finished


0.746451860375911

In [30]:
test_values = pd.read_csv('../../csv/test_values.csv', index_col = "building_id")
test_values

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,t,r,q,...,1,0,0,0,0,0,0,0,0,0
663567,10,1407,11907,3,25,6,7,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1049160,22,1136,7712,1,50,3,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [31]:
test_values_subset = test_values
test_values_subset["geo_level_1_id"] = test_values_subset["geo_level_1_id"].astype("category")
test_values_subset

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
300051,17,596,11307,3,20,7,6,t,r,n,...,0,0,0,0,0,0,0,0,0,0
99355,6,141,11987,2,25,13,5,t,r,n,...,1,0,0,0,0,0,0,0,0,0
890251,22,19,10044,2,5,4,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
745817,26,39,633,1,0,19,3,t,r,x,...,0,0,1,0,0,0,0,0,0,0
421793,17,289,7970,3,15,8,7,t,r,q,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,4,605,3623,3,70,20,6,t,r,q,...,1,0,0,0,0,0,0,0,0,0
663567,10,1407,11907,3,25,6,7,n,r,n,...,0,0,0,0,0,0,0,0,0,0
1049160,22,1136,7712,1,50,3,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [32]:
test_values_subset.shape

(86868, 38)

In [33]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

features_to_encode = ["geo_level_1_id", "land_surface_condition", "foundation_type", "roof_type",\
                     "position", "ground_floor_type", "other_floor_type",\
                     "plan_configuration", "legal_ownership_status"]
for feature in features_to_encode:
    test_values_subset = encode_and_bind(test_values_subset, feature)
test_values_subset

,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,...,plan_configuration_m,plan_configuration_n,plan_configuration_o,plan_configuration_q,plan_configuration_s,plan_configuration_u,legal_ownership_status_a,legal_ownership_status_r,legal_ownership_status_v,legal_ownership_status_w
building_id,,,,,,,,,,,,,,,,,,,,,
300051,596,11307,3,20,7,6,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
99355,141,11987,2,25,13,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
890251,19,10044,2,5,4,5,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
745817,39,633,1,0,19,3,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
421793,289,7970,3,15,8,7,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310028,605,3623,3,70,20,6,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
663567,1407,11907,3,25,6,7,1,1,1,0,...,0,0,0,0,0,0,0,0,1,0
1049160,1136,7712,1,50,3,3,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [34]:
# Genero las predicciones para los test.
preds = eclf_model.predict(test_values_subset)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    3.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.2min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.2min finished


In [35]:
submission_format = pd.read_csv('../../csv/submission_format.csv', index_col = "building_id")

In [36]:
my_submission = pd.DataFrame(data=preds,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [37]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3


In [38]:
my_submission.to_csv('../../csv/predictions/jf/ensemble-vote/jf-model-3-submission.csv')

In [39]:
!head ../../csv/predictions/jf/ensemble-vote/jf-model-3-submission.csv

building_id,damage_grade
300051,3
99355,2
890251,2
745817,1
421793,3
871976,2
691228,2
896100,3
343471,2
